In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from os import listdir
import re

import csv
import datetime
import pickle
import os

import calendar
import random

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.options.display.float_format = '{:.0f}'.format

In [3]:
DATA_ROOT_PATH="/mnt/data/projects/MD1/data/R1000/reports/"
PROJ_ROOT_PATH="/home/priyesh/projects/MD1"

In [4]:
# Read from Pickle file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','clean_master_data_stage2.pkl')
clean_master_data = pd.read_pickle(filepath)

In [5]:
# Read from Pickle file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','master_data_stage2.pkl')
master_data = pd.read_pickle(filepath)

In [6]:
# Read from Pickle file

filepath = os.path.join(PROJ_ROOT_PATH,'pickle','mappings_balance_stage3.pkl')
label_mappings = pd.read_pickle(filepath)

In [7]:
heading_lookup = {}
line_item_lookup = {}

stype='balance'

temp = label_mappings['headings'].to_dict()
heading_lookup[stype] = temp['type']

temp = label_mappings['line_items'].to_dict()
line_item_lookup[stype] = temp['type']

In [8]:
def find_line_items(table, heading_lookup, line_item_lookup, stype):
 
  df_new = pd.DataFrame()

  h = 'No Heading'
  items_found = []
    
  h_lookup = heading_lookup[stype]
  l_lookup = line_item_lookup[stype]
    
  items_required = list(set(l_lookup.values()))

  # Go through each line in table, if row contains only nan then assume heading

  for index, row in table.iterrows():

    if row.isnull().all():
   
      # If all columns are null then it is either a header we are interested in, a header not recognised 
      # or a line item where values are not specified.
    
      # If we encounter a header that has Member or Abstract then terminate loop and ignore subsequent items.
    
      if index in h_lookup:
        
        h = h_lookup[index]
        
        if h == 'EXCL':
          break
        else:
          r1 = row.copy()
          #r1 = r1.rename(swap_text('H',index,stype))
          r1['type'] = h
          r1['line_type'] = 'H'
          df_new = pd.concat([df_new, pd.DataFrame([r1])])
  
    else:
      # If row matches one of the items mapped for current header then store
        
      if index in l_lookup:
        
        l = l_lookup[index]
        items_found.append(l)
          
        r1 = row.copy()
        #r1.rename(swap_text('L',index,stype))
        r1['type'] = l
        r1['line_type'] = 'L'
        df_new = pd.concat([df_new, pd.DataFrame([r1])])
    
    if re.search('Total Liabilities And.*Equity$',index,re.IGNORECASE): 
        break
        
    if re.search('Total Liabilities, Temporary Equity And Permanent Equity$',index,re.IGNORECASE): 
        break
 
  return(df_new)

In [9]:
def extract_summary(stype):
    
  ticker_list = list(clean_master_data.keys())

  master_statements = {}

  for ticker in ticker_list:

    master_statements[ticker] = {}
    print(ticker)
    
    for yr in clean_master_data[ticker]:
    
      # Extract statement for each year
    
      master_statements[ticker]
      if stype in clean_master_data[ticker][yr]:
        
        master_statements[ticker][yr] = find_line_items(clean_master_data[ticker][yr][stype]['table'],
                                                          heading_lookup,line_item_lookup,stype)
  return master_statements

In [10]:
def conv_date_cols(cols):
    
  mnth_lookup = {month: index for index, month in enumerate(calendar.month_abbr) if month}

  new_cols = []

  for c in cols:
    
    x = re.search(r"^[A-Z][a-z][a-z]\. \d{2}, \d{4}", c)
    
    if x:
      mnth = mnth_lookup[c.split(".")[0]]
      yr = c[-4:]
      t = yr + '-' + str(mnth)
      new_cols = new_cols + [t]
    else:
      new_cols = new_cols + [c] 

  return new_cols

In [11]:
def find_duplicates(df1):

  df_temp = df1.copy()
  df_temp = df_temp.reset_index(names=['line_item'])
  df_dup = df_temp[df_temp.duplicated(subset=['line_item','line_type'])]

  return df_dup

In [12]:
def extract_statements(stype, master_summary_data):
    
  ticker_list = list(master_summary_data.keys())

  master_statements = {}

  for ticker in master_summary_data:
    
    print(ticker)
    statement = master_summary_data[ticker]
    keys = list(statement.keys())
    
    if len(keys) > 0:
                                            
      keys = sorted(keys,reverse=True)
      df = statement[keys[0]].copy()
      df = df[~df.index.duplicated(keep='first')]

      for i in range(1,len(keys)):
        
        print("Building Table for",ticker,i)
        
        df1_cols = list(set(statement[keys[i]].columns)- set(df.columns))
        df1 = statement[keys[i]][df1_cols].copy()
        df1 = df1[~df1.index.duplicated(keep='first')] 
        df = df.merge(df1, right_index=True, left_index=True, how='left', suffixes=['_x', '_y'])  
    
      # We could end up with an empty dataframe if there are no matches. So skip if that is the case.
     
      if len(df) > 0:
        # Amend date format of column names
        
        cols = list(df.columns)
        new_cols = conv_date_cols(cols)
        df.columns = new_cols

        # Sort in reverse date order, place type and line_type at end
    
        new_cols.remove('type')
        new_cols.remove('line_type')
        cols = sorted(new_cols,reverse=True) + ['type','line_type']
        df = df[cols]
        
      master_statements[ticker] = df

  return master_statements

In [13]:
df1 = clean_master_data['BK']['18']['balance']['table']
df1

,"Dec. 31, 2017","Dec. 31, 2016"
line_item,,
Securities:,NaN,NaN
"Held-To-Maturity Fair Value Of 40,512 And 40,669",40827,NaN
Loans,61540,64458
Allowance For Loan Losses,-159,-169
Goodwill,17564,17316
Intangible Assets,3411,3598
"Other Assets Includes 791 And 1,339, At Fair Value",23029,20954
Total Assets,371758,333469
Deposits:,NaN,NaN


In [14]:
find_line_items(df1, heading_lookup, line_item_lookup, 'balance')


,"Dec. 31, 2017","Dec. 31, 2016",type,line_type
Goodwill,17564,17316,IA,L
Intangible Assets,3411,3598,IA,L
Total Assets,371758,333469,TA,L
Long-Term Debt,27979,24463,NCD,L
Total Liabilities,330012,293889,TL,L
Temporary Equity,NaN,NaN,SE,H
Permanent Equity,NaN,NaN,SE,H
Retained Earnings,25635,22621,RE,L
Total Company Shareholders Equity,41251,38811,TSE,L
Total Permanent Equity,41567,39429,TSE,L


In [15]:
# Create lookup dictionaries

swap_text_dict = {}

swap_text_dict['H'] = {}
swap_text_dict['L'] = {}

In [16]:
balance_summary = extract_summary('balance')

AA
AAL
AAP
AAPL
ABBV
ABNB
ABT
ACGL
ACHC
ACI
ACM
ACN
ADBE
ADI
ADM
ADP
ADSK
ADT
AEE
AEP
AES
AFG
AFL
AFRM
AGCO
AGL
AGNC
AGO
AGR
AIG
AIZ
AJG
AKAM
AL
ALB
ALGM
ALGN
ALK
ALL
ALLE
ALLY
ALNY
ALSN
AM
AMAT
AMC
AMCR
AMD
AME
AMED
AMG
AMGN
AMP
AMZN
AN
ANET
ANSS
AON
AOS
APA
APD
APH
APO
APP
APTV
AR
ARES
ARMK
ARW
ASH
ATO
ATR
ATUS
ATVI
AVGO
AVT
AVTR
AWI
AWK
AXON
AXS
AXTA
AYI
AYX
AZEK
AZO
AZTA
BA
BAC
BAH
BALL
BAX
BBWI
BBY
BC
BDX
BEN
BERY
BFAM
BG
BHF
BIIB
BILL
BIO
BJ
BK
BKNG
BLD
BLDR
BLK
BMRN
BMY
BOKF
BR
BRKR
BRO
BSX
BSY
BURL
BWA
BWXT
BX
BYD
C
CABO
CACC
CAH
CAR
CARR
CAT
CB
CBOE
CBSH
CC
CCCS
CCK
CCL
CDAY
CDNS
CDW
CE
CEG
CERT
CF
CFG
CFLT
CFR
CG
CGNX
CHD
CHDN
CHE
CHH
CHPT
CHRW
CHTR
CIEN
CINF
CL
CLH
CLVT
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNA
CNM
CNP
CNXC
COF
COIN
COLB
COLM
COO
COST
COTY
CPB
CPRI
CPRT
CR
CRI
CRL
CRUS
CRWD
CSCO
CSL
CTLT
CTRA
CTSH
CTVA
CVNA
CVX
CW
CZR
D
DAL
DAR
DASH
DBX
DCI
DD
DDOG
DECK
DFS
DG
DGX
DHI
DHR
DINO
DIS
DISH
DKNG
DKS
DLB
DLTR
DNA
DOCS
DOCU
DOV
DOW
DPZ
DRVN
DT
DTE
DTM
DUK
DV
DVA
DVN
DXC
DX

In [17]:
len(balance_summary)

833

In [18]:
balance_summary['NFG']['22']

,"Sep. 30, 2022","Sep. 30, 2021",type,line_type
Assets,NaN,NaN,CA,H
"Property, Plant And Equipment, Net, Total",6566477,6384283,NPPE,L
Current Assets,NaN,NaN,CA,H
Total Current Assets,761131,522449,TCA,L
Goodwill,5476,5476,IA,L
Total Assets,7896262,7464825,TA,L
Total Company Shareholders Equity,2079896,1786206,TSE,L
Current And Accrued Liabilities,NaN,NaN,CL,H
Interest Payable On Long-Term Debt,17376,17376,NCD,L
Total Current And Accrued Liabilities,1942569,1236133,TCL,L


In [19]:
dups_list = []
tickers_with_dups = {}

ticker_list = list(balance_summary.keys())

for ticker in ticker_list:
  
  print(ticker)
    
  for yr in balance_summary[ticker]:
    
    df_dup = find_duplicates(balance_summary[ticker][yr])
    
    if len(df_dup) > 0:
       
       dups = {}
       dups['yr'] = yr
       dups['df_orig'] = balance_summary[ticker][yr]
       dups['df_dup']  = df_dup
        
       tickers_with_dups[ticker] = dups


AA
AAL
AAP
AAPL
ABBV
ABNB
ABT
ACGL
ACHC
ACI
ACM
ACN
ADBE
ADI
ADM
ADP
ADSK
ADT
AEE
AEP
AES
AFG
AFL
AFRM
AGCO
AGL
AGNC
AGO
AGR
AIG
AIZ
AJG
AKAM
AL
ALB
ALGM
ALGN
ALK
ALL
ALLE
ALLY
ALNY
ALSN
AM
AMAT
AMC
AMCR
AMD
AME
AMED
AMG
AMGN
AMP
AMZN
AN
ANET
ANSS
AON
AOS
APA
APD
APH
APO
APP
APTV
AR
ARES
ARMK
ARW
ASH
ATO
ATR
ATUS
ATVI
AVGO
AVT
AVTR
AWI
AWK
AXON
AXS
AXTA
AYI
AYX
AZEK
AZO
AZTA
BA
BAC
BAH
BALL
BAX
BBWI
BBY
BC
BDX
BEN
BERY
BFAM
BG
BHF
BIIB
BILL
BIO
BJ
BK
BKNG
BLD
BLDR
BLK
BMRN
BMY
BOKF
BR
BRKR
BRO
BSX
BSY
BURL
BWA
BWXT
BX
BYD
C
CABO
CACC
CAH
CAR
CARR
CAT
CB
CBOE
CBSH
CC
CCCS
CCK
CCL
CDAY
CDNS
CDW
CE
CEG
CERT
CF
CFG
CFLT
CFR
CG
CGNX
CHD
CHDN
CHE
CHH
CHPT
CHRW
CHTR
CIEN
CINF
CL
CLH
CLVT
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNA
CNM
CNP
CNXC
COF
COIN
COLB
COLM
COO
COST
COTY
CPB
CPRI
CPRT
CR
CRI
CRL
CRUS
CRWD
CSCO
CSL
CTLT
CTRA
CTSH
CTVA
CVNA
CVX
CW
CZR
D
DAL
DAR
DASH
DBX
DCI
DD
DDOG
DECK
DFS
DG
DGX
DHI
DHR
DINO
DIS
DISH
DKNG
DKS
DLB
DLTR
DNA
DOCS
DOCU
DOV
DOW
DPZ
DRVN
DT
DTE
DTM
DUK
DV
DVA
DVN
DXC
DX

In [20]:
tickers_with_dups.keys()

dict_keys(['ACGL', 'ACM', 'ACN', 'AFRM', 'AGCO', 'AGL', 'AGO', 'AGR', 'AIZ', 'ALGM', 'AM', 'AMCR', 'APO', 'APTV', 'ARES', 'ATR', 'ATUS', 'AXS', 'AXTA', 'BAH', 'BALL', 'BEN', 'BILL', 'BIO', 'BRKR', 'BYD', 'CAH', 'CCL', 'CE', 'CHD', 'CHDN', 'CHTR', 'CL', 'CLX', 'CMI', 'CNM', 'COIN', 'COTY', 'CPRI', 'CRL', 'CRWD', 'CTLT', 'CTVA', 'CVX', 'D', 'DAR', 'DDOG', 'DGX', 'DHR', 'DISH', 'DLB', 'DNA', 'DRVN', 'DTM', 'EEFT', 'EFX', 'EHAB', 'EHC', 'EIX', 'ERIE', 'ES', 'ETR', 'EVA', 'EXC', 'EXPE', 'FE', 'FNF', 'FTV', 'GE', 'GILD', 'GWW', 'HAL', 'HAYW', 'HEI', 'HIG', 'HLT', 'HOLX', 'HON', 'HPE', 'HPP', 'HPQ', 'HSIC', 'HSY', 'HXL', 'IAC', 'ICE', 'ICUI', 'IDA', 'IDXX', 'ILMN', 'INTC', 'IONS', 'IP', 'IPG', 'IR', 'ISRG', 'ITT', 'IVZ', 'J', 'JAZZ', 'JHG', 'JNPR', 'KBR', 'KKR', 'KLAC', 'KMB', 'KMI', 'KNX', 'KSS', 'LAD', 'LAZ', 'LDOS', 'LKQ', 'LLY', 'LNG', 'LVS', 'LYV', 'M', 'MA', 'MAN', 'MAR', 'MCO', 'MDB', 'MGM', 'MHK', 'MRO', 'MRTX', 'MSA', 'MSGS', 'MSI', 'MSM', 'MTCH', 'MTN', 'MTZ', 'NCLH', 'NCR', 'NET', 

In [21]:
len(tickers_with_dups)

219

In [22]:
ticker='BIO'
yr = tickers_with_dups[ticker]['yr']
tickers_with_dups[ticker]['df_dup']

,line_item,"Dec. 31, 2012","Dec. 31, 2011",type,line_type
11,Total Company Shareholders Equity,2011270,1744382,TSE,L


In [ ]:
tickers_with_dups[ticker]['df_orig']

In [23]:
df1 = clean_master_data[ticker][yr]['balance']['table']
df1

,"Dec. 31, 2012","Dec. 31, 2011"
line_item,,
Assets:,NaN,NaN
Cash And Cash Equivalents,463388,574231
Short-Term Investments,457685,238884
Accounts Receivables Net Of Allowances,398739,398674
Inventories:,NaN,NaN
Raw Materials,93009,99326
Work In Process,124737,120191
Finished Goods,230624,213993
Total Inventories,448370,433510


In [24]:
master_data[ticker][yr]['balance']

,"Consolidated Balance Sheets (USD $) In Thousands, unless otherwise specified","Dec. 31, 2012","Dec. 31, 2011"
0,ASSETS:,NaN,NaN
1,Cash and cash equivalents,"$ 463,388","$ 574,231"
2,Short-term investments,457685,238884
3,"Accounts receivable, less allowance for doubtful accounts",398739,398674
4,Inventories:,NaN,NaN
5,Raw materials,93009,99326
6,Work in process,124737,120191
7,Finished goods,230624,213993
8,Total inventories,448370,433510
9,Deferred tax assets,57751,53777


In [25]:
# Remove companies with duplicates

for ticker in tickers_with_dups.keys():
   print(ticker)
   clean_master_data.pop(ticker)

ACGL
ACM
ACN
AFRM
AGCO
AGL
AGO
AGR
AIZ
ALGM
AM
AMCR
APO
APTV
ARES
ATR
ATUS
AXS
AXTA
BAH
BALL
BEN
BILL
BIO
BRKR
BYD
CAH
CCL
CE
CHD
CHDN
CHTR
CL
CLX
CMI
CNM
COIN
COTY
CPRI
CRL
CRWD
CTLT
CTVA
CVX
D
DAR
DDOG
DGX
DHR
DISH
DLB
DNA
DRVN
DTM
EEFT
EFX
EHAB
EHC
EIX
ERIE
ES
ETR
EVA
EXC
EXPE
FE
FNF
FTV
GE
GILD
GWW
HAL
HAYW
HEI
HIG
HLT
HOLX
HON
HPE
HPP
HPQ
HSIC
HSY
HXL
IAC
ICE
ICUI
IDA
IDXX
ILMN
INTC
IONS
IP
IPG
IR
ISRG
ITT
IVZ
J
JAZZ
JHG
JNPR
KBR
KKR
KLAC
KMB
KMI
KNX
KSS
LAD
LAZ
LDOS
LKQ
LLY
LNG
LVS
LYV
M
MA
MAN
MAR
MCO
MDB
MGM
MHK
MRO
MRTX
MSA
MSGS
MSI
MSM
MTCH
MTN
MTZ
NCLH
NCR
NET
NFE
NI
NOV
NVST
NXST
NYT
OGE
OKTA
ON
OSH
OWL
PATH
PCAR
PDCE
PEG
PEN
PENN
PEP
PFE
PG
PGR
PINC
PINS
PLNT
PLTK
PM
PPC
PPG
PRGO
PSX
QCOM
RARE
RH
RNR
RSG
RUN
S
SCHW
SF
SMAR
SNA
SNDR
SNOW
SNPS
SPB
STT
STZ
SYK
TDOC
TDY
TECH
TMUS
TOST
TPX
TRGP
TRMB
TRU
TSLA
TSN
TWKS
TXT
UBER
UHS
VFC
VIRT
VMI
VMW
VNT
VOYA
VRSK
VRTX
VST
WAB
WCC
WDAY
WEX
WHR
WMS
WSO
WYNN
XEL
YUM


In [26]:
# Repeat the extraction. This time we shouldn't have any duplicates.

balance_summary = extract_summary('balance')

AA
AAL
AAP
AAPL
ABBV
ABNB
ABT
ACHC
ACI
ADBE
ADI
ADM
ADP
ADSK
ADT
AEE
AEP
AES
AFG
AFL
AGNC
AIG
AJG
AKAM
AL
ALB
ALGN
ALK
ALL
ALLE
ALLY
ALNY
ALSN
AMAT
AMC
AMD
AME
AMED
AMG
AMGN
AMP
AMZN
AN
ANET
ANSS
AON
AOS
APA
APD
APH
APP
AR
ARMK
ARW
ASH
ATO
ATVI
AVGO
AVT
AVTR
AWI
AWK
AXON
AYI
AYX
AZEK
AZO
AZTA
BA
BAC
BAX
BBWI
BBY
BC
BDX
BERY
BFAM
BG
BHF
BIIB
BJ
BK
BKNG
BLD
BLDR
BLK
BMRN
BMY
BOKF
BR
BRO
BSX
BSY
BURL
BWA
BWXT
BX
C
CABO
CACC
CAR
CARR
CAT
CB
CBOE
CBSH
CC
CCCS
CCK
CDAY
CDNS
CDW
CEG
CERT
CF
CFG
CFLT
CFR
CG
CGNX
CHE
CHH
CHPT
CHRW
CIEN
CINF
CLH
CLVT
CMA
CMCSA
CME
CMG
CMS
CNA
CNP
CNXC
COF
COLB
COLM
COO
COST
CPB
CPRT
CR
CRI
CRUS
CSCO
CSL
CTRA
CTSH
CVNA
CW
CZR
DAL
DASH
DBX
DCI
DD
DECK
DFS
DG
DHI
DINO
DIS
DKNG
DKS
DLTR
DOCS
DOCU
DOV
DOW
DPZ
DT
DTE
DUK
DV
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EL
ELAN
EMN
EMR
ENOV
ENPH
ENTG
EOG
EPAM
EQH
EQT
ESAB
ESI
ESTC
ETN
ETSY
EVR
EVRG
EW
EWBC
EXAS
EXEL
EXP
EXPD
F
FAF
FANG
FAST
FCN
FCNCA
FCX
FDS
FFIV
FG
FHB
FHN
FICO
FIS
FISV
FITB
FIVE
FIVN
FLO
FLS
FLT
FMC
FNB
FND
FOUR
F

In [27]:
key_list = list(balance_summary.keys())
len(key_list)

614

In [47]:
key_list = list(balance_summary.keys())
i = random.randint(0,len(key_list))
ticker=key_list[i]
print(ticker)
balance_summary[ticker]['21']

BAC


,"Dec. 31, 2020","Dec. 31, 2019",type,line_type
Assets,NaN,NaN,CA,H
Cash And Cash Equivalents,380463,161560,CCE,L
Goodwill,68951,68951,IA,L
Total Assets,2819627,2434079,TA,L
Short-Term Debt,19321,24204,CD,L
Long-Term Debt,262934,240856,NCD,L
Total Liabilities,2546703,2169269,TL,L
Shareholders Equity,NaN,NaN,SE,H
Retained Earnings,164088,156319,RE,L
Total Company Shareholders Equity,272924,264810,TSE,L


In [39]:
print(ticker)
clean_master_data[ticker]['22']['balance']['table']

ENTG


,"Dec. 31, 2021","Dec. 31, 2020"
line_item,,
Current Assets:,NaN,NaN
Cash And Cash Equivalents,402565,580893
"Trade Accounts And Notes Receivable, Net",347413,264392
"Inventories, Net",475213,323944
Deferred Tax Charges And Refundable Income Taxes,35312,21136
Other Current Assets,52867,43892
Total Current Assets,1313370,1234257
Property Plant And Equipment,654098,525367
Other Assets:,NaN,NaN


In [ ]:
ticker = 'PPL'
yr=tickers_with_dups[ticker]['yr']
print(yr)
tickers_with_dups[ticker]['df_dup']

In [ ]:

df1 = clean_master_data[ticker][yr]['balance']['table']
df1

In [ ]:
master_data[ticker][yr]['cash']

In [ ]:
df1 = clean_master_data[ticker][yr]['balance']['table']
find_line_items(df1, heading_lookup, line_item_lookup, 'balance')

In [ ]:
balance_summary[ticker][yr]

In [ ]:
find_duplicates(balance_summary[ticker][yr])

### Build Statements

In [48]:
master_statements = extract_statements('balance',balance_summary)

AA
Building Table for AA 1
Building Table for AA 2
Building Table for AA 3
Building Table for AA 4
Building Table for AA 5
Building Table for AA 6
AAL
Building Table for AAL 1
Building Table for AAL 2
Building Table for AAL 3
Building Table for AAL 4
Building Table for AAL 5
Building Table for AAL 6
Building Table for AAL 7
Building Table for AAL 8
Building Table for AAL 9
Building Table for AAL 10
Building Table for AAL 11
AAP
Building Table for AAP 1
Building Table for AAP 2
Building Table for AAP 3
Building Table for AAP 4
Building Table for AAP 5
Building Table for AAP 6
Building Table for AAP 7
Building Table for AAP 8
Building Table for AAP 9
Building Table for AAP 10
AAPL
Building Table for AAPL 1
Building Table for AAPL 2
Building Table for AAPL 3
Building Table for AAPL 4
Building Table for AAPL 5
Building Table for AAPL 6
Building Table for AAPL 7
Building Table for AAPL 8
Building Table for AAPL 9
Building Table for AAPL 10
Building Table for AAPL 11
ABBV
Building Table for 

Building Table for AN 9
Building Table for AN 10
Building Table for AN 11
ANET
Building Table for ANET 1
Building Table for ANET 2
Building Table for ANET 3
Building Table for ANET 4
Building Table for ANET 5
Building Table for ANET 6
Building Table for ANET 7
Building Table for ANET 8
ANSS
Building Table for ANSS 1
Building Table for ANSS 2
Building Table for ANSS 3
Building Table for ANSS 4
Building Table for ANSS 5
Building Table for ANSS 6
Building Table for ANSS 7
Building Table for ANSS 8
Building Table for ANSS 9
Building Table for ANSS 10
Building Table for ANSS 11
AON
Building Table for AON 1
Building Table for AON 2
Building Table for AON 3
Building Table for AON 4
Building Table for AON 5
Building Table for AON 6
Building Table for AON 7
Building Table for AON 8
Building Table for AON 9
AOS
Building Table for AOS 1
Building Table for AOS 2
Building Table for AOS 3
Building Table for AOS 4
Building Table for AOS 5
Building Table for AOS 6
Building Table for AOS 7
Building Tab

Building Table for BURL 5
Building Table for BURL 6
Building Table for BURL 7
Building Table for BURL 8
Building Table for BURL 9
BWA
Building Table for BWA 1
Building Table for BWA 2
Building Table for BWA 3
Building Table for BWA 4
Building Table for BWA 5
Building Table for BWA 6
Building Table for BWA 7
Building Table for BWA 8
Building Table for BWA 9
Building Table for BWA 10
Building Table for BWA 11
BWXT
Building Table for BWXT 1
Building Table for BWXT 2
Building Table for BWXT 3
Building Table for BWXT 4
Building Table for BWXT 5
Building Table for BWXT 6
Building Table for BWXT 7
Building Table for BWXT 8
Building Table for BWXT 9
Building Table for BWXT 10
BX
Building Table for BX 1
Building Table for BX 2
Building Table for BX 3
Building Table for BX 4
Building Table for BX 5
Building Table for BX 6
Building Table for BX 7
Building Table for BX 8
Building Table for BX 9
Building Table for BX 10
Building Table for BX 11
C
Building Table for C 1
Building Table for C 2
Buildi

Building Table for CPRT 7
Building Table for CPRT 8
Building Table for CPRT 9
Building Table for CPRT 10
CR
Building Table for CR 1
Building Table for CR 2
Building Table for CR 3
Building Table for CR 4
Building Table for CR 5
Building Table for CR 6
Building Table for CR 7
Building Table for CR 8
Building Table for CR 9
Building Table for CR 10
Building Table for CR 11
CRI
Building Table for CRI 1
Building Table for CRI 2
Building Table for CRI 3
Building Table for CRI 4
Building Table for CRI 5
Building Table for CRI 6
Building Table for CRI 7
Building Table for CRI 8
Building Table for CRI 9
Building Table for CRI 10
Building Table for CRI 11
CRUS
Building Table for CRUS 1
Building Table for CRUS 2
Building Table for CRUS 3
Building Table for CRUS 4
Building Table for CRUS 5
Building Table for CRUS 6
Building Table for CRUS 7
Building Table for CRUS 8
Building Table for CRUS 9
Building Table for CRUS 10
CSCO
Building Table for CSCO 1
Building Table for CSCO 2
Building Table for CSC

Building Table for ETSY 6
EVR
Building Table for EVR 1
Building Table for EVR 2
Building Table for EVR 3
Building Table for EVR 4
Building Table for EVR 5
Building Table for EVR 6
Building Table for EVR 7
Building Table for EVR 8
Building Table for EVR 9
Building Table for EVR 10
EVRG
Building Table for EVRG 1
Building Table for EVRG 2
Building Table for EVRG 3
Building Table for EVRG 4
EW
Building Table for EW 1
Building Table for EW 2
Building Table for EW 3
Building Table for EW 4
Building Table for EW 5
Building Table for EW 6
Building Table for EW 7
Building Table for EW 8
Building Table for EW 9
Building Table for EW 10
Building Table for EW 11
EWBC
Building Table for EWBC 1
Building Table for EWBC 2
Building Table for EWBC 3
Building Table for EWBC 4
Building Table for EWBC 5
Building Table for EWBC 6
Building Table for EWBC 7
Building Table for EWBC 8
Building Table for EWBC 9
Building Table for EWBC 10
Building Table for EWBC 11
EXAS
Building Table for EXAS 1
Building Table fo

...

Building Table for NLY 7
Building Table for NLY 8
Building Table for NLY 9
Building Table for NLY 10
Building Table for NLY 11
NOC
Building Table for NOC 1
Building Table for NOC 2
Building Table for NOC 3
Building Table for NOC 4
Building Table for NOC 5
Building Table for NOC 6
Building Table for NOC 7
Building Table for NOC 8
Building Table for NOC 9
Building Table for NOC 10
Building Table for NOC 11
NOW
Building Table for NOW 1
Building Table for NOW 2
Building Table for NOW 3
Building Table for NOW 4
Building Table for NOW 5
Building Table for NOW 6
Building Table for NOW 7
Building Table for NOW 8
Building Table for NOW 9
NRG
Building Table for NRG 1
Building Table for NRG 2
Building Table for NRG 3
Building Table for NRG 4
Building Table for NRG 5
Building Table for NRG 6
Building Table for NRG 7
Building Table for NRG 8
Building Table for NRG 9
Building Table for NRG 10
Building Table for NRG 11
NSC
Building Table for NSC 1
Building Table for NSC 2
Building Table for NSC 3
Bui

Building Table for PLUG 4
Building Table for PLUG 5
Building Table for PLUG 6
Building Table for PLUG 7
Building Table for PLUG 8
Building Table for PLUG 9
Building Table for PLUG 10
Building Table for PLUG 11
PNC
Building Table for PNC 1
Building Table for PNC 2
Building Table for PNC 3
Building Table for PNC 4
Building Table for PNC 5
Building Table for PNC 6
Building Table for PNC 7
Building Table for PNC 8
Building Table for PNC 9
Building Table for PNC 10
Building Table for PNC 11
PNFP
Building Table for PNFP 1
Building Table for PNFP 2
Building Table for PNFP 3
Building Table for PNFP 4
Building Table for PNFP 5
Building Table for PNFP 6
Building Table for PNFP 7
Building Table for PNFP 8
Building Table for PNFP 9
Building Table for PNFP 10
Building Table for PNFP 11
PNR
Building Table for PNR 1
Building Table for PNR 2
Building Table for PNR 3
Building Table for PNR 4
Building Table for PNR 5
Building Table for PNR 6
Building Table for PNR 7
Building Table for PNR 8
Building Tab

SJM
Building Table for SJM 1
Building Table for SJM 2
Building Table for SJM 3
Building Table for SJM 4
Building Table for SJM 5
Building Table for SJM 6
Building Table for SJM 7
Building Table for SJM 8
Building Table for SJM 9
Building Table for SJM 10
Building Table for SJM 11
SKX
Building Table for SKX 1
Building Table for SKX 2
Building Table for SKX 3
Building Table for SKX 4
Building Table for SKX 5
Building Table for SKX 6
Building Table for SKX 7
Building Table for SKX 8
Building Table for SKX 9
Building Table for SKX 10
Building Table for SKX 11
SLB
Building Table for SLB 1
Building Table for SLB 2
Building Table for SLB 3
Building Table for SLB 4
Building Table for SLB 5
Building Table for SLB 6
Building Table for SLB 7
Building Table for SLB 8
Building Table for SLB 9
Building Table for SLB 10
Building Table for SLB 11
SLGN
Building Table for SLGN 1
Building Table for SLGN 2
Building Table for SLGN 3
Building Table for SLGN 4
Building Table for SLGN 5
Building Table for SLG

Building Table for TPR 7
Building Table for TPR 8
Building Table for TPR 9
Building Table for TPR 10
TREX
Building Table for TREX 1
Building Table for TREX 2
Building Table for TREX 3
Building Table for TREX 4
Building Table for TREX 5
Building Table for TREX 6
Building Table for TREX 7
Building Table for TREX 8
Building Table for TREX 9
Building Table for TREX 10
TRIP
Building Table for TRIP 1
Building Table for TRIP 2
Building Table for TRIP 3
Building Table for TRIP 4
Building Table for TRIP 5
Building Table for TRIP 6
Building Table for TRIP 7
Building Table for TRIP 8
Building Table for TRIP 9
Building Table for TRIP 10
Building Table for TRIP 11
TROW
Building Table for TROW 1
Building Table for TROW 2
Building Table for TROW 3
Building Table for TROW 4
Building Table for TROW 5
Building Table for TROW 6
Building Table for TROW 7
Building Table for TROW 8
Building Table for TROW 9
Building Table for TROW 10
TRV
Building Table for TRV 1
Building Table for TRV 2
Building Table for T

Building Table for WFC 3
Building Table for WFC 4
Building Table for WFC 5
Building Table for WFC 6
Building Table for WFC 7
Building Table for WFC 8
Building Table for WFC 9
Building Table for WFC 10
Building Table for WFC 11
WH
Building Table for WH 1
WLK
Building Table for WLK 1
Building Table for WLK 2
Building Table for WLK 3
Building Table for WLK 4
Building Table for WLK 5
Building Table for WLK 6
Building Table for WLK 7
Building Table for WLK 8
Building Table for WLK 9
Building Table for WLK 10
Building Table for WLK 11
WM
Building Table for WM 1
Building Table for WM 2
Building Table for WM 3
Building Table for WM 4
Building Table for WM 5
Building Table for WM 6
Building Table for WM 7
WMB
Building Table for WMB 1
Building Table for WMB 2
Building Table for WMB 3
Building Table for WMB 4
Building Table for WMB 5
Building Table for WMB 6
Building Table for WMB 7
Building Table for WMB 8
Building Table for WMB 9
Building Table for WMB 10
Building Table for WMB 11
WMT
Building 

In [49]:
master_statements['AAPL']

,2022-9,2021-9,2020-9,2019-9,2018-9,2017-9,2016-9,2015-9,2014-9,2013-9,2012-9,2011-9,2010-9,type,line_type
Current Assets:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA,H
Cash And Cash Equivalents,23646,34940,38016,48844,25913,20289,20484,21120,13844,14259,10746,9815,11261,CCE,L
Total Current Assets,135405,134836,143713,162819,131339,128645,106869,89378,68531,73286,57653,44988,41678,TCA,L
Total Assets,352755,351002,323888,338516,365725,375319,321686,290345,231839,207000,176064,116371,75183,TA,L
Current Liabilities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CL,H
Total Current Liabilities,153982,125481,105392,105718,115929,100814,79006,80610,63448,43658,38542,27970,20722,TCL,L
Total Non-Current Liabilities,148101,162431,153157,142310,142649,140458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TNCL,L
Total Liabilities,302083,287912,258549,248028,258578,241272,193437,170990,120292,83451,57854,39756,27392,TL,L
Shareholders Equity:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SE,H
Retained Earnings/-Accumulated Deficit,-3068,5562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RE,L


In [63]:
key_list = list(balance_summary.keys())
i = random.randint(0,len(key_list))
ticker=key_list[i]
print(ticker)
master_statements[ticker]

TDC


,2022-12,2021-12,2020-12,2019-12,2018-12,2017-12,2016-12,2015-12,2014-12,2013-12,2012-12,2011-12,2010-12,type,line_type
Current Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA,H
Cash And Cash Equivalents,569,592,529,494,715,1089,974,839,834,695,729,772,883,CCE,L
Total Current Assets,1028,1106,1044,1014,1428,1750,1621,1734,1572,1563,1534,1412,1406,TCA,L
Goodwill,390,396,401,396,395,399,390,380,948,946,932,742,136,IA,L
Total Assets,2022,2169,2193,2057,2360,2556,2413,2527,3132,3096,3066,2616,1883,TA,L
Current Liabilities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CL,H
Total Current Liabilities,1007,1033,952,886,1009,1063,729,953,995,776,806,695,569,TCL,L
Long-Term Debt,498,324,411,454,478,478,538,567,195,248,274,290,0,NCD,L
Total Liabilities,1764,1709,1793,1795,1865,1888,1442,1678,1425,1239,1287,1122,694,TL,L
Company Shareholders Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SE,H
